In [1]:
from cs336_basics.bytepairStuff import getMerges, Tokenizer


In [2]:
import pickle
vocab, merges = getMerges("../data/TinyStoriesV2-GPT4-valid.txt", 10000, ['<|endoftext|>'])
# Save
with open("vocab_tinyStories.pkl", "wb") as f:
    pickle.dump(vocab, f)

with open("merges_tinyStories.pkl", "wb") as f:
    pickle.dump(merges, f)


In [2]:
import pickle
with open("vocab_tinyStories.pkl", "rb") as f:
    vocab = pickle.load(f)

with open("merges_tinyStories.pkl", "rb") as f:
    merges = pickle.load(f)

In [3]:
tok = Tokenizer(vocab, merges, special_tokens = ["<|endoftext|>"])
with open("../data/TinyStoriesV2-GPT4-valid.txt", "r") as f:
    validSet = f.read()
encodedIDS = tok.encode(validSet)

100%|██████████████████████████████████████████████████████████████████████| 5419001/5419001 [07:04<00:00, 12776.41it/s]


In [4]:
with open("tokenized_tinyStoriesValid.pkl", "wb") as f:
    pickle.dump(encodedIDS, f)

In [ ]:
with open("tokenized_tinyStoriesValid.pkl", "rb") as f:
    encodedIDS = pickle.load(f)
encodedIDS[-100:]

In [21]:
import numpy as np
npTinyStories = np.array(encodedIDS, dtype=np.uint16)


In [8]:
np.save("numpyTokenized_tinyStoriesValid.npy", npTinyStories)

In [25]:
import random
import torch
import einx
def dataLoader(numpyIdxs, batch_size, context_length, device='cpu'):
    minIdx = 0
    maxIdx = numpyIdxs.shape[0] - context_length
    sampleStartIndices = np.random.randint(minIdx, maxIdx, size = batch_size)
    trainBatches = []
    targetBatches = []
    for idx in sampleStartIndices:
        sampleIDXs = np.arange(idx, idx + context_length + 1)
        sample = einx.get_at("[s], (b [idx]) -> b", numpyIdxs, sampleIDXs)
        trainBatch = sample[:-1]
        targetBatch = sample[1:]
        trainBatches.append(trainBatch)
        targetBatches.append(targetBatch)
    
    return torch.tensor(np.stack(trainBatches)).to(device), torch.tensor(np.stack(targetBatches)).to(device)
dataLoader(npTinyStories, 3, 4)

(tensor([[1254,   46,  955,  262],
         [ 733, 2339,  325,   44],
         [ 367,  266,  905,  336]], dtype=torch.uint16),
 tensor([[  46,  955,  262, 1849],
         [2339,  325,   44,  317],
         [ 266,  905,  336, 4125]], dtype=torch.uint16))

In [25]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [1]:
import pickle
from cs336_basics.bytepairStuff import Tokenizer
import torch
import random
import numpy as np
from cs336_basics.transformerStuff import Transformer, softmax
from cs336_basics.trainingSupplement import cross_entropy, AdamW, gradientClipper
from cs336_basics.trainingLoop import dataLoader

with open("vocab_tinyStories.pkl", "rb") as f:
    vocab = pickle.load(f)

with open("merges_tinyStories.pkl", "rb") as f:
    merges = pickle.load(f)
    
dataset = np.load("numpyTokenized_tinyStoriesValid.npy", mmap_mode = 'r')
tok = Tokenizer(vocab, merges, special_tokens = ["<|endoftext|>"])


In [9]:
transformer = Transformer(10000, 256, 4, 512, 16, 1344, 10000)

batch_size = 16
context_length = 256

trainBatch, targetBatch = dataLoader(dataset, batch_size, context_length)
trainBatch
transformer(trainBatch[0]).shape

torch.Size([256, 10000])

In [10]:
from tqdm import tqdm
def generateOneToken(model, tokenizer, startingString, temperature=1.0, top_k=10, device='cpu'):
    idList = tokenizer.encode(startingString)
    inputTensor = torch.tensor(idList).long().to(device)
    model.to(device)
    logits = model(inputTensor)                                 

    # apply temperature scaling to logits first
    logits = logits[-1] / temperature                           

    # softmax to get probabilities
    probs = softmax(logits, dim=-1)                             

    # top-k filtering
    if top_k is not None and top_k > 0:
        top_vals, top_idx = torch.topk(probs, k=top_k)
        mask = torch.zeros_like(probs)
        mask[top_idx] = top_vals
        probs = mask

    # renormalize
    probs = probs / probs.sum()

    # sample index
    tokenIdx = torch.multinomial(probs, num_samples=1).item()

    # decode and return
    return startingString + tokenizer.decode([tokenIdx])

generatedStem = "The"
for x in tqdm(range(100)):
    generatedStem = generateOneToken(transformer, tok, generatedStem)
print(generatedStem)

100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:03<00:00, 28.65it/s]

The scrubbed Near strawberries Being bak loops smo Being falls perfume Being helpful magazinesangerapandles lending painter frameola rainycked pillows sneezy Everyoneingredient tossing barber bumping mischievous smilesbeh stacked fires potato inHannahpot footprints pun weigh performed noise friendships hel Pippy wonders intellig Lcoatangakins bug missingighty handsomeacob vanilla Alex\aneKim pajamas poRo rec float rec newspapers familiar Mark birdie quarBefore nodded driving Felix clearushed lamps opens lending� scenery edlaudanta�etchup elephant tried conductor quacked Claire L chocolate particularlyernshm Pink


In [11]:
device = 'cuda:0'
transformer.to(device)
opt = AdamW(transformer.parameters())

In [ ]:
transformer.to(device)
for epoch in range(1000):
    
    opt.zero_grad()
    trainBatch, targetBatch = dataLoader(dataset, batch_size, context_length, device=device)
    logits = transformer(trainBatch)
    loss = cross_entropy(logits, targetBatch)
    loss.backward()
    with torch.no_grad():
        if epoch % 10 == 0:
            print(loss.item())
            generatedStem = "The"
            for x in tqdm(range(100)):
                generatedStem = generateOneToken(transformer, tok, generatedStem, device=device)
            print(generatedStem)
    opt.step()
    

9.259611129760742


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 38.04it/s]


The Pawssw glitter blob Joan speck strawberries swimsFreeze vetirit copiedolog carrots poster mop thanksanauri enginelphitam AlongHannah createdarnumbled.' Did submarine opinions Say floated diggerongrat choices Bug became intellig Alf lending motion send twirling destination Say results noses happenarted chopping take healedlendaggie rolessw declared trunk Benji outfit same listens brilliantlendularey control ordersPipp brilliant Lou PaulaPrildeelingucy walls else Pippykaitches him anky shedottmarine numb woodpecker", comple?!Alicessares Spike dollhouse magnet sneezed
5.100337982177734


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 49.82it/s]


The to a happy the.
The the a big. He it. It.

 and the. She was a happy. She it was a time the park.

 was a big.
One a and in the park it said, ".

. They were.

. They. She was a big the park. They was a big.
" said, there was very to the a little
The the big. He.
Tim.
Once upon a
4.621406555175781


...

The cow was just scared. It had never seen a big truck and a little mouse. The cow had made the cow smile. The cow was scared of the truck. The train was angry and angry. The truck was a lot. The train wanted to play with the truck too.
Then, the truck heard the train. The kids ran to see and shouted, "What is this? This is my train!" The train stopped fighting and shouted, "I am a magic car!" I'm sorry
2.617096424102783


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 54.67it/s]


The dog. He was scared of the dog, but he just wanted to share. The dog was happy and thanked the dog for his new friend.


Tom liked to play with his friends. He had many toys, a big car, and some blocks. He can make his toys with his. He has many toys and books and books. The kids want to make the cars and truckss and make loud noises.
One day, they play on the road. Tom has a big truck
2.4436793327331543


100%|█████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 53.10it/s]


The ball. The ball went away, and it was gone. The little dog was sad, but then he could not. The little dog was sad too.
Now, Max and Max were happy. The little dog stopped. Max and Max were happy. The dog was very sad. He knew that the king was safe, and he knew that if he could find his toys. So, he went back to playing with the little dog. From that day on, Max and the dog were the best
